In [ ]:
import time
import random
import numpy as np
import torch
import json

import messages
from simulator import Simulator
from agent import Agent
from unity_env import UnityEnv

In [ ]:
simulator = Simulator(port=5004)

In [ ]:
env = UnityEnv(simulator, 9, 2, 1, lambda: 2*np.random.random((2,))-1, "SimpleUnityEnv")
agent = Agent(env)

In [ ]:
cp = torch.load(f"save/{env.get_name()}/checkpoint.pt")
agent.policy.load_state_dict(cp['pi'])
agent.q1.load_state_dict(cp['q1'])
agent.q2.load_state_dict(cp['q2'])
agent.replay_buffer = cp["replay_buffer"]
from_epoch = cp['epoch']

In [ ]:
simulator.start_server()
serv = simulator.server

In [ ]:
env.set_time_scale(20)

In [ ]:
agent.train(from_epoch=0, epoch_len=2000, start_steps=4000)

In [ ]:
obs = env.step((0,0))

In [ ]:
feat = agent.policy.net(obs)
mu = agent.policy.mu_layer(feat)
log_std = agent.policy.log_std_layer(feat)
std = torch.exp(log_std)

mu, std, log_std

In [ ]:
msg = messages.RequestMessage("test", "").to_json()
msg2 = messages.RequestMessage("timeScale", messages.SingleFieldMessage(2.).to_json()).to_json()
msg3 = messages.RequestMessage("pause", messages.SingleFieldMessage(False).to_json()).to_json()
msg4 = messages.RequestMessage("reset", "").to_json()
msg5 = messages.RequestMessage("step", messages.ControllMessage(5, 5).to_json()).to_json()


In [ ]:
r = lambda d: {k:round(v, 5) for (k, v) in d.items()}

print(r(simulator._reset().velocity))
time.sleep(env.get_target_frame_duration())
print(r(simulator._step(10, 1).velocity))
for _ in range(3):
    #time.sleep(env.get_target_frame_duration())
    print(r(simulator._step(0, 1).velocity))
simulator.shutdown()
simulator.server.clients = []

In [ ]:
serv.stop_server()

In [ ]:
simulator.shutdown()
simulator.server.clients = []

In [ ]:
simulator.server.clients = []